In [1]:
import json
import pandas as pd
import requests
import urllib
import datetime

# Import google maps data

In [19]:
df = pd.read_csv('Low-Cost Sensor Locations- Deployed Sensors.csv')

# Extract floating values using regular expressions
df[['longitude', 'latitude']] = df['WKT'].str.extract(r'POINT \((\d+\.\d+) (\d+\.\d+)\)')

# Convert columns to float
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

#Dropping WKT column
df = df.drop('WKT', axis=1)

# Create a new DataFrame for the transformed rows
df_transformed = pd.concat([df, df])
df_transformed = df_transformed.reset_index(drop=True)

# Update the Sensor values in the transformed DataFrame
half_rows = int(len(df_transformed) // 2)
df_transformed.loc[half_rows:, ['SENSOR_1', 'SENSOR_2']] = df_transformed.loc[half_rows:, ['SENSOR_2', 'SENSOR_1']].values

# Reset the index of the transformed DataFrame
df_transformed = df_transformed.reset_index(drop=True)

df_transformed.head()


,SITE_ID,SENSOR_1,SENSOR_2,DEPLOYMENT,DESCRIPTION,LAMP_ID,longitude,latitude
0,LOTC,13126.0,13172.0,1.683025e+15,Co-Location LfU Lothstrasse,NaN,11.554620,48.154567
1,HANV,13171.0,13147.0,1.683023e+15,NaN,1.0,11.528536,48.172732
2,DONV,13092.0,13096.0,1.683027e+15,NaN,505.0,11.537630,48.159656
3,LANC,13101.0,13114.0,1.683032e+15,Co-Location LfU Landshuter Allee,23.0,11.536546,48.149570
4,FRIV,13118.0,13143.0,1.683034e+15,NaN,10.0,11.519285,48.143002


In [68]:
# append elevation to the dataframe
# from https://gis.stackexchange.com/questions/338392/getting-elevation-for-multiple-lat-long-coordinates-in-python

url = 'https://api.opentopodata.org/v1/eudem25m?'

def make_remote_request(url: str, params: dict):
   """
   Makes the remote request
   Continues making attempts until it succeeds
   """

   count = 1
   while True:
       try:
           response = requests.get((url + urllib.parse.urlencode(params)))
       except (OSError, urllib3.exceptions.ProtocolError) as error:
           print('\n')
           print('*' * 20, 'Error Occured', '*' * 20)
           print(f'Number of tries: {count}')
           print(f'URL: {url}')
           print(error)
           print('\n')
           count += 1
           continue
       break
   return response


def elevation_function(latitude, longitude):
    """Query service using lat, lon. add the elevation values as a new column."""
    params = {'locations': f"{latitude},{longitude}"}
    result = make_remote_request(url, params)
    return result.json()['results'][0]['elevation']

# append elevation to dataframe
df_transformed['elev'] = df_transformed.apply(lambda row: elevation_function(row['latitude'], row['longitude']), axis = 1)


# Build SITE.json

### SITES 
    
    'DUEB': {
        site_type: 'reference_colocation',
        site_lat: '000',
        site_lon: '000',
        site_elevation: '000',
        comment: ''
        }

#### possible *site_types*
- individual
- reference_colocation
- lfu_colocation
- midcost_colocation
- lc_aq_colocation
- traffic_count

In [86]:
site_ids = df_transformed['SITE_ID'].unique()

sites = {}

for id in site_ids: 
    site_info = df_transformed.set_index('SITE_ID').loc[id].iloc[0]
    description = site_info['DESCRIPTION']
    description = site_info['DESCRIPTION']
    dct = {'site_type': site_info['DESCRIPTION'],
           'site_lat': site_info['latitude'],
           'site_lon': site_info['longitude'],
           'elevation': int(site_info['elev']), 
           'comment': 'Lamp post ids:' + str(site_info['LAMP_ID'])
           }
    
    sites.update({id:dct})

In [87]:
json_object = json.dumps(sites, indent=4)
with open("SITES.json", "w") as outfile:
    outfile.write(json_object)


# Build SAMPLING.json

### SAMPLING

    [{
        site_id: 'DUEB',
        sensor_ids : [0],
        sampling_start: '2022-03-01', 
        sampling_end : '2022-03-12',
        orientation: '230',
        elevation_ag: 3.25,
        comment: ''
    },]

#### options for *sensor_ids*
- [0] : all sensors in SENSORS.json
- [13077] (example): Sensor with sensor_id 13077
- [13077,13136]: Sensor 13077 and 13136  

In [95]:
df_transformed.head()

,SITE_ID,SENSOR_1,SENSOR_2,DEPLOYMENT,DESCRIPTION,LAMP_ID,longitude,latitude,elev
0,LOTC,13126.0,13172.0,1.683025e+15,Co-Location LfU Lothstrasse,NaN,11.554620,48.154567,519.894958
1,HANV,13171.0,13147.0,1.683023e+15,NaN,1.0,11.528536,48.172732,510.212830
2,DONV,13092.0,13096.0,1.683027e+15,NaN,505.0,11.537630,48.159656,520.449524
3,LANC,13101.0,13114.0,1.683032e+15,Co-Location LfU Landshuter Allee,23.0,11.536546,48.149570,524.841125
4,FRIV,13118.0,13143.0,1.683034e+15,NaN,10.0,11.519285,48.143002,523.496765


In [110]:
site_ids = df_transformed['SITE_ID'].unique()

sampling = []

for id in site_ids:
       site_info = df_transformed.set_index('SITE_ID').loc[id].iloc[0]
       try: 
              sens_1 = int(site_info['SENSOR_1'])
              sens_2 = int(site_info['SENSOR_2'])
              deployment = datetime.datetime.fromtimestamp(site_info['DEPLOYMENT']/1000000).strftime("%Y-%m-%dT%H:%M+02:00")
              
              dct = {"site_id": id, 
                     "sensor_ids": [sens_1,sens_2],
                     "sampling_start": deployment, 
                     "sampling_end": "",
                     "orientation": "0",
                     "elevation_ag": "3",
                     "comment": ""}
              sampling.append(dct)
       except: 
              print(id)

TUMC


In [112]:
json_object = json.dumps(sampling, indent=4)
with open("SAMPLING.json", "w") as outfile:
    outfile.write(json_object)


In [113]:
with open('SAMPLING.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)